# 一 进程原理

进程在执行的过程中要实现快速的切换，这是为了能够给人一种看起来好像在同时进行的样子的体验，与此同时，我们要考虑的是，这个进程切换过去了等会肯定还要在切换回来，那么再切换回来的时候，这个进程还是原来的样子吗？只有当进程还保持原来的样子这样的切换才算是有意义的，否则在切换之前CPU的工作岂不是白白浪费了吗？
>所以，并发的本质其实就是：切换+保存状态

# 二 进程的创建
你双击运行你的机器上任何一个程序都是起了一个进程，右击运行你写的Python代码也是起了一个进程。它的本质其实就是向操作系统申请一块内存空间，然后把数据放进去，这样一个进程就起来了。

# 三 并发工作原理
原来我们按照软件开发规范写的程序，假如里面都有2个任务，但是启动文件只有一个，所以，当你启动程序的时候，其实就是起了一个进程，也就是把这一个进程给了操作系统。如果我们能够同时启动10个同样的进程，把这10个进程都交给操作系统，操作系统就会在这10个进程之间来回切换，这也就实现了并发。如果是四核CPU来处理这10个进程，那么同一时间还会有四个进程可以实现并行。但是把同一个程序运行10次手动开启10个进程，这种方式肯定不是我们所期望的，最好是我们能够调用操作系统的一个接口，在执行第一个任务的时候由操作系统来自动的开启一个进程，同时执行第二个任务，也就是把我们程序中的两个任务放到不同的进程中去，操作系统就会在这两个进程之间来回切换。这两个进程是父子关系，首先运行的是第一个进程，由第一个进程自动的向操作系统申请一块内存空间，把第一个进程所产生的数据放到这个内存空间中，CPU到这个内存空间取数据，这样第二个进程也就起来了，也就是开启了一个子进程，对于操作系统而言，这两个进程都是进程，那么他就会在这两个进程之间切换，这样就能实现并发。
创建新的进程主要有4种方式：
>1 系统初始化
2 一个进程在运行过程中开启子进程
3 用户的交互式请求
4 批处理作业的初始化

我们主要研究的就是第二种：一个进程在运行过程中开启子进程。
其实这样的例子我们在上文中已经出现了，在执行Pycharm这个进程的时候又开启了一个Python进程，在执行终端这个进程的时候也是又开启了一个Python进程。很明显开启一个父进程，他的数据是来源于硬盘，而开启一个子进程它的数据是来源于内存。
对于创建子进程来说UNIX系统（MacOS系统和Linux系统都是UNIX系统内核）和Windows系统是有所区别的，UNIX系统会从自己父进程的内存空间完整的拷贝一份给子进程，作为子进程运行的初始状态，所以子进程的初始状态和父进程是一模一样的，但是只要是不同的进程，他们的内存空间在硬件级别就是隔离的。 随着子进程与父进程的运行，假如子进程把数据修改了，对于父进程没有任何影响。Windows系统创建子进程也会拷贝一份父进程内存空间的数据，但是子进程还会初始化一些自己独有的数据，所以，子进程的初始状态并不是和父进程一模一样的。这两者的区别对于我们写程序没有影响，只是他们底层的工作原理不同而已。
只要是开启一个进程就是向操作系统发送请求，调用操作系统的功能，其实也就是调用它的接口，操作系统一般是基于C语言和少量的汇编语言编写的，其实底层调用机器硬件的都是C语言的接口，UNIX系统上调用的新建进程的接口叫做 fork，而Windows系统调用的新建进程的接口叫做 CreateProcess，我们的Python语言是基于C语言编写的，所以我们所使用的Python接口其实就是调用了龟叔用C语言给我们封装好的接口。

# 四 开启子进程的两种方式
## 1. 函数
把子进程的任务写成函数，并当作参数传递给多进程类
>


In [2]:
from multiprocessing import Process
import time
def task(name):  # 如果不传参数，下面调用类的时候args=(...)去掉就可以了
    print('2 %s进程开始' % name)
    time.sleep(3)
    print('3 %s进程结束' % name)
# **在Windows系统上，开启子进程的操作一定要放在这下面操作**
if __name__ == '__main__':
    # target参数指定要开启的进程所要执行的任务，args后面的数据类型必须是元祖
    p = Process(target=task, args=('Albert',))
    # p = Process(target=task, kwargs={'name':'Albert'})  # 这两种方式都可以
    p.start()  # 向操作系统发指令，申请内存空间，拷贝数据，创建子进程
    print('1 这是主进程')
    


1 这是主进程


p.start()是在主进程运行中给操作系统发了一个信号，发完信号他就继续执行下面的代码，但是对于操作系统而言，要经历一系列过程才能创建这个子进程，所以你会看到我标识的1，2，3的打印顺序，虽然1和2几乎是在同时完成，但他们依然是有先后顺序的。根据这个打印结果，我们可以判断出父进程和子进程在操作系统上的执行是并发执行的，而不是按照我们代码的先后顺序执行的。
由于Windows系统开进程的操作会重新再导入一遍父进程的文件，这也就是说第一次运行到p.start()这行代码的时候，会把以上代码重新倒入，第二次在执行p.start()也是一样的，所以如果你不写那个分支语句就相当于是无限导入，当我们写了这个分支语句__> _name___就不再等于main，而是等于这个模块名，所以分支的条件不成立，就可以正常使用了。因此，在Windows系统上开子进程一定要写在main下面，MacOS系统随意。
## 2. 类
重写多进程类

In [3]:
from multiprocessing import Process
import time
class MyProcess(Process):
    def __init__(self, name):  # 重写父类init
        super(Process, self).__init__()  # 重用父类init
        self.name = name
    def run(self):  # 重写run方法，把原来的任务写到这里面去
        print('2 %s进程开始' % self.name)
        time.sleep(3)
        print('3 %s进程结束' % self.name)
if __name__ == '__main__':
    p = MyProcess('Albert')  # 可以传参数，自然也可以不传参数，由你掌控
    p.start()  # 调用的就是run方法
    print('1 这是主进程')

1 这是主进程


这里我们重写Process类，我们并不是十分清楚这个类里面都有哪些东西，所以需要注意重用父类的功能，另外p.start()最后调用的是run这个方法，所以，一定要记住，run方法这个名字不能变。有兴趣的同学可以去看一下，Python这一部分的源码。
![avator](https://cdn.nlark.com/yuque/0/2019/png/281865/1554539216562-addd38d6-3fca-4747-a510-48daa10311f3.png#align=left&display=inline&height=384&originHeight=322&originWidth=720&size=0&status=done&width=859)


In [9]:
ticket={
    '1':'50',
    '2': '52',
    '3': '53',
    '4': '54',
    '5': '55',
    '6': '56',
    '7': '57',
    '8': '58',
    '9': '59',
    '10': '60'
}
def check_ticket():
    return len(ticket)
def buy_ticket():
    print(ticket)
    choice=input('输入要购买的车票序号')
    if choice in ticket:
        ticket.pop(choice)
    return ticket
buy_ticket()
check_ticket()

{'1': '50', '2': '52', '3': '53', '4': '54', '5': '55', '6': '56', '7': '57', '8': '58', '9': '59', '10': '60'}
输入要购买的车票序号1


9

In [10]:
print(ticket)

{'2': '52', '3': '53', '4': '54', '5': '55', '6': '56', '7': '57', '8': '58', '9': '59', '10': '60'}


# 互斥锁
假如现在有三个任务都在不同的软件中运行，他们都有自己的一篇内容需要打印

In [11]:
from multiprocessing import Process
import time, random
# 假如这个进程是由word程序打印
def task1():
    print('任务1 名字是：英格拉姆')
    time.sleep(random.randint(1, 5))
    print('任务1 性别是：male')
    time.sleep(random.randint(1, 5))
    print('任务1 年龄是：22')
    time.sleep(random.randint(1, 5))
# 这个进程是由notepad++程序打印
def task2():
    print('任务2 名字是：卡戴珊')
    time.sleep(random.randint(1, 5))  # 模拟IO操作
    print('任务2 性别是：female')
    time.sleep(random.randint(1, 5))
    print('任务2 年龄是：25')
    time.sleep(random.randint(1, 5))
# 这个进程是由wps程序打印
def task3():
    print('任务3 名字是：詹姆斯')
    time.sleep(random.randint(1, 5))
    print('任务3 性别是：male')
    time.sleep(random.randint(1, 5))
    print('任务3 年龄是：34')
    time.sleep(random.randint(1, 5))
if __name__ == '__main__':
    p1 = Process(target=task1, )
    p2 = Process(target=task2, )
    p3 = Process(target=task3, )
    p1.start()
    p2.start()
    p3.start()
    print('主进程')

主进程


由于三个进程是并发执行的，所以操作系统控制打印来回在多个任务之间切换，最后你看到的打印结果就错乱了。 你是更想要效率还是更想要正确的结果？打印机是怎么工作的？怎么样这个任务就不会乱了？
所以，我们要做的就是把并发变成串行，才能保证不出现错乱。

In [ ]:
from multiprocessing import Process
import time, random
# 假如这个进程是由word程序打印
def task1():
    print('任务1 名字是：英格拉姆')
    time.sleep(random.randint(1, 5))  # 模拟IO操作
    print('任务1 性别是：male')
    time.sleep(random.randint(1, 5))
    print('任务1 年龄是：22')
    time.sleep(random.randint(1, 5))
# 这个进程是由notepad++程序打印
def task2():
    print('任务2 名字是：卡戴珊')
    time.sleep(random.randint(1, 5))
    print('任务2 性别是：female')
    time.sleep(random.randint(1, 5))
    print('任务2 年龄是：25')
    time.sleep(random.randint(1, 5))
# 这个进程是由wps程序打印
def task3():
    print('任务3 名字是：詹姆斯')
    time.sleep(random.randint(1, 5))
    print('任务3 性别是：male')
    time.sleep(random.randint(1, 5))
    print('任务3 年龄是：34')
    time.sleep(random.randint(1, 5))
if __name__ == '__main__':
    p1 = Process(target=task1, )
    p2 = Process(target=task2, )
    p3 = Process(target=task3, )
    p1.start()
    p1.join()
    p2.start()
    p2.join()
    p3.start()
    p3.join()
    print('主进程')

但是，我们要做的是并发，所谓的并发就是所有的子进程几乎在同时创建，那么就有个可能p3这个子进程先启动起来了而不是p1，但我们现在的操作是人为的把子进程按照先后的顺序执行， 显然这么做是不合理的。应该是谁先抢这个资源就由谁先使用，而且是直到第一个使用者使用结束，后面的使用者再继续争抢，以此循环，这就像是大学一个寝室的同学使用洗手间一样，谁先进去就一定会上一把锁在门上，直到他出来才会打开这个锁，其他人再进去的时候，只要拿到了这个锁，那么就掌握了主动权。所以说，要保证后面的人进不去，执行进程的关键其实就在于这把锁。

In [ ]:
from multiprocessing import Process, Lock
import time, random
mutex = Lock()  # 实例化锁的对象 
# 假如这个进程是由word程序打印
def task1(lock):
    lock.acquire()  # 上锁
    print('任务1 名字是：英格拉姆')
    time.sleep(random.randint(1, 5))
    print('任务1 性别是：male')
    time.sleep(random.randint(1, 5))
    print('任务1 年龄是：22')
    time.sleep(random.randint(1, 5))
    lock.release() # 解锁 
# 这个进程是由notepad++程序打印
def task2(lock):
    lock.acquire()  # 如果连续两次抢锁，第一次抢到了，接着抢一定抢不到，那么程序就卡了
    # lock.acquire()
    print('任务2 名字是：卡戴珊')
    time.sleep(random.randint(1, 5))
    print('任务2 性别是：female')
    time.sleep(random.randint(1, 5))
    print('任务2 年龄是：25')
    time.sleep(random.randint(1, 5))
    lock.release()
# 这个进程是由wps程序打印
def task3(lock):
    lock.acquire()
    print('任务3 名字是：詹姆斯')
    time.sleep(random.randint(1, 5))
    print('任务3 性别是：male')
    time.sleep(random.randint(1, 5))
    print('任务3 年龄是：34')
    time.sleep(random.randint(1, 5))
    lock.release()
if __name__ == '__main__':
    p1 = Process(target=task1, args=(mutex,))
    p2 = Process(target=task2, args=(mutex,))
    p3 = Process(target=task3, args=(mutex,))
    p1.start()
    p2.start()
    p3.start()
    print('主进程')

注意：每个子进程抢的都是同一把锁，但是子进程之间的内存空间又是完全隔离的，所以我们只能通过参数的形式把这把锁传递给子进程。你看到的结果就是只要有任何一个进程先起来了，那么就一定会等这个进程的任务执行完了其他的进程才能进来。此时程序的执行是三个进程都起来了，但为了保证打印内容的准确，依然是把并发变成了串行，这一点与join类似。与区别在于：join是人为的规定好顺序， 现在是所有的进程有公平竞争的机会。 这把锁就叫做互斥锁，所有的进程相互排斥，同一时间只能由一个进程掌控。

#  IPC机制
IPC(Inter Process Communication)机制就是进程间的通信，进程之间内存是隔离的，不能直接通信，我们知道的可以用一个中间介质比如硬盘来完成这个过程，但是这样的效率太低了。硬盘之所以能够解决是因为硬盘上的文件是可以共享的， 但是如果我们能够在内存留一个所有进程共享的内存空间，那么这个问题就可以完美的解决了 。确实有这样一个内存空间，但是！！！我要说的是：紧接着我要讲的东西了解就好了，我不会深入说明，因为这个设计有缺陷，你如果用了，极有可能会给你的程序带来bug，所以知道有这个东西就好了。
>IPC机制需要三件事：
    1 找一块所拥有进程共享的空间
    2 这个空间必须是内存空间
    3 帮我们自动处理好锁的问题
 
**这个机制就是Queue，他指的是队列，队列是先进先出。它造出来的对象就能符合我们以上三点。**

In [12]:
from multiprocessing import Queue
q = Queue(3)  # 参数3指的是排队的有3个人
q.put("1")  # 三个人一次排队
q.put("2")
q.put("3")
print(q.get())
print(q.get())
print(q.get())
# print(q.get())  # 只有三个人排队，取不出来第四个人，程序卡死

1
2
3


需要注意的是：
第一点：队列用来存储进程间沟通的消息，数据量不应该过大。就像是我作为老板在北京给你打电话，要求你去盖一个房子，我只是打电话告诉你就好了，不会把钢筋水泥都寄给你。所以队列之间的通信又常常被人们称为消息队列。
第二点：刚才进程之间通信的参数我们设置成3，但是如果你写成300000这是没有意义的，这会局限于你的内存的限制。
put有两个默认参数，是block=True和timeout=None，分别指的是阻塞和超时时间。

In [13]:
from multiprocessing import Queue
q = Queue(3)
q.put("1", block=True)  # 这样表示默认会阻塞
q.put("2", block=True)
q.put("3", block=True)
q.put("4", block=True, timeout=3)  # 3秒之后结束阻塞，程序报错，如果没有timeout就会一直阻塞

Full: 

In [14]:
from multiprocessing import Queue
q = Queue(3)
q.put("1", block=False)  # 前三个都不会阻塞，True或者False都无所谓
q.put("2", block=True)
q.put("3", block=True)
q.put("4", block=False)  # 不阻塞直接报错

Full: 



# 生产者消费者模型
从这个模型你就能够看出来，这是一种写程序的思路。该模型包含两类重要的角色：
>
1 生产者：将负责造数据的任务比喻为生产者。
2 消费之：将负责处理生产者造出来的数据的任务比喻为消费者。

在生活这样的例子非常常见
肯德基的快餐之所以快就是因为厨师只负责生产食物，然后将生产好的食物放在吧台，而作为消费者只需要负责吃就好了，两者之间互补影响。
如果是我们之前的工作方式，同时进行的程序只能有一个，要么吃，要么生产，但是现在我们有了多进程，这个任务可以同时进行，在程序的执行中，再一次实现了程序的解耦合。

实现生产者消费者模型的三要素：

生产者
消费者
队列
注意：生产者消费者模型只是一种解决问题或者写代码的思路，没有明确表示一定是用什么技术。
什么时候用该模型？
程序中出现明显的两类任务，一类任务是负责生产数据，另一类任务是负责处理生产的数据，最明显的例子就是爬虫和数据分析的组合。
用该模型有什么好处？

实现了生产者与消费者解耦和
平衡了生产力与消费力，即生产者可以一直不停地生产，消费者可以不停地处理，因为二者不再直接沟通的，而是跟队列沟通